In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer, KNNImputer
import os
from datetime import datetime
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
print("✅ All libraries imported successfully!")
print(f"📅 Data preprocessing started on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
data_path = '../data/raw/dataset.csv'
output_path = '../data/processed/'
try:
    df_raw = pd.read_csv(data_path)
    print("✅ Raw dataset loaded successfully!")
    print(f"📊 Original dataset shape: {df_raw.shape[0]:,} rows × {df_raw.shape[1]} columns")
    print(f"💾 Memory usage: {df_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
except FileNotFoundError:
    print("❌ Dataset file not found. Please check the file path.")
except Exception as e:
    print(f"❌ Error loading dataset: {e}")
df = df_raw.copy()
print(f"\n🔄 Working copy created for preprocessing")

In [ ]:
print("🔍 INITIAL DATA ASSESSMENT")
print("=" * 50)
print(f"Dataset Shape: {df.shape}")
print(f"\nColumn Information:")
for i, col in enumerate(df.columns, 1):
    missing_count = df[col].isnull().sum()
    missing_pct = (missing_count / len(df)) * 100
    print(f"{i:2d}. {col:<25} | Missing: {missing_count:3d} ({missing_pct:5.1f}%)")
print(f"\n📊 Total Missing Values: {df.isnull().sum().sum():,}")
print(f"📊 Data Completeness: {((df.shape[0] * df.shape[1] - df.isnull().sum().sum()) / (df.shape[0] * df.shape[1]) * 100):.1f}%")

In [ ]:
print("🧹 STEP 1: BASIC DATA CLEANING")
print("=" * 40)
initial_shape = df.shape
duplicate_rows = df.duplicated().sum()
print(f"Duplicate rows found: {duplicate_rows}")
if duplicate_rows > 0:
    df = df.drop_duplicates()
    print(f"✅ Removed {duplicate_rows} duplicate rows")
print(f"\n📋 Company Symbol Analysis:")
if 'Symbol' in df.columns:
    duplicate_symbols = df['Symbol'].duplicated().sum()
    print(f"Duplicate symbols: {duplicate_symbols}")
    if duplicate_symbols > 0:
        print("🔍 Duplicate symbols found:")
        duplicated_syms = df[df['Symbol'].duplicated(keep=False)]['Symbol'].value_counts()
        print(duplicated_syms)
    else:
        print("✅ All company symbols are unique")
print(f"\n📊 Shape after basic cleaning: {df.shape} (removed {initial_shape[0] - df.shape[0]} rows)")

In [ ]:
print("🧹 STEP 2: EMPLOYEE DATA STANDARDIZATION")
print("=" * 45)
if 'Full Time Employees' in df.columns:
    print("Processing employee count data...")
    print(f"Original data type: {df['Full Time Employees'].dtype}")
    
    def clean_employee_count(value):
        if pd.isna(value):
            return np.nan
        if isinstance(value, (int, float)):
            return float(value)
        
        value_str = str(value).strip()
        if value_str == '' or value_str.lower() in ['nan', 'null', 'none']:
            return np.nan
        
        value_clean = re.sub(r'[^0-9.]', '', value_str)
        try:
            return float(value_clean)
        except:
            return np.nan
    
    df['Full Time Employees'] = df['Full Time Employees'].apply(clean_employee_count)
    
    employee_stats = df['Full Time Employees'].describe()
    print(f"✅ Employee data standardized:")
    print(f"   Valid records: {df['Full Time Employees'].count()}/{len(df)}")
    print(f"   Range: {employee_stats['min']:,.0f} to {employee_stats['max']:,.0f}")
    print(f"   Median: {employee_stats['50%']:,.0f}")
    
    extreme_outliers = df['Full Time Employees'] > 1000000
    if extreme_outliers.any():
        print(f"\n⚠️  Large companies (>1M employees):")
        large_companies = df[extreme_outliers][['Symbol', 'Name', 'Full Time Employees']]
        for _, row in large_companies.iterrows():
            print(f"   {row['Symbol']}: {row['Full Time Employees']:,.0f} employees")
else:
    print("❌ 'Full Time Employees' column not found")

In [ ]:
print("🧹 STEP 3: ESG RISK SCORES PREPROCESSING")
print("=" * 45)
esg_columns = []
for col in df.columns:
    if any(keyword in col.lower() for keyword in ['esg', 'risk', 'score', 'environment', 'social', 'governance']):
        if df[col].dtype in ['int64', 'float64'] or col in ['Total ESG Risk score', 'Environment Risk Score', 'Governance Risk Score', 'Social Risk Score', 'Controversy Score']:
            esg_columns.append(col)
print(f"🎯 Identified {len(esg_columns)} ESG-related numeric columns:")
for i, col in enumerate(esg_columns, 1):
    print(f"   {i}. {col}")
if esg_columns:
    print(f"\n📊 ESG Data Completeness Analysis:")
    for col in esg_columns:
        valid_count = df[col].count()
        missing_count = df[col].isnull().sum()
        missing_pct = (missing_count / len(df)) * 100
        print(f"   {col:<25}: {valid_count:3d}/{len(df)} valid ({100-missing_pct:5.1f}%)")
    
    print(f"\n🔍 ESG Score Ranges:")
    for col in esg_columns:
        if df[col].count() > 0:
            min_val = df[col].min()
            max_val = df[col].max()
            mean_val = df[col].mean()
            print(f"   {col:<25}: {min_val:6.1f} - {max_val:6.1f} (mean: {mean_val:6.1f})")
else:
    print("❌ No ESG columns identified")

In [ ]:
print("🧹 STEP 4: CATEGORICAL DATA STANDARDIZATION")
print("=" * 45)
categorical_columns = ['Sector', 'Industry', 'Controversy Level', 'ESG Risk Level', 'ESG Risk Percentile']
for col in categorical_columns:
    if col in df.columns:
        print(f"\n📊 Processing {col}:")
        
        df[col] = df[col].astype(str).str.strip()
        df[col] = df[col].replace(['nan', 'None', 'null', ''], np.nan)
        
        unique_values = df[col].value_counts(dropna=False)
        print(f"   Unique values: {len(unique_values)} (including NaN)")
        print(f"   Missing: {df[col].isnull().sum()} ({(df[col].isnull().sum()/len(df)*100):.1f}%)")
        
        if col == 'ESG Risk Percentile':
            print(f"   Sample values: {list(unique_values.head().index)}")
            
            def extract_percentile_number(value):
                if pd.isna(value) or value == 'nan':
                    return np.nan
                try:
                    percentile_match = re.search(r'(\d+)', str(value))
                    if percentile_match:
                        return int(percentile_match.group(1))
                    return np.nan
                except:
                    return np.nan
            
            df['ESG_Risk_Percentile_Numeric'] = df[col].apply(extract_percentile_number)
            valid_percentiles = df['ESG_Risk_Percentile_Numeric'].dropna()
            if len(valid_percentiles) > 0:
                print(f"   ✅ Created numeric percentile column: {valid_percentiles.min():.0f}-{valid_percentiles.max():.0f} range")
        
        elif col in ['Controversy Level', 'ESG Risk Level']:
            print(f"   Top categories: {list(unique_values.head().index)}")
        
        elif col in ['Sector', 'Industry']:
            print(f"   Top 5 categories: {list(unique_values.head().index)}")
    else:
        print(f"   ❌ {col} not found in dataset")

In [ ]:
print("🧹 STEP 5: MISSING VALUE TREATMENT STRATEGY")
print("=" * 50)
missing_analysis = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df)) * 100,
    'Data_Type': df.dtypes
}).sort_values('Missing_Percentage', ascending=False)
high_missing = missing_analysis[missing_analysis['Missing_Percentage'] > 10]
medium_missing = missing_analysis[(missing_analysis['Missing_Percentage'] > 1) & (missing_analysis['Missing_Percentage'] <= 10)]
low_missing = missing_analysis[(missing_analysis['Missing_Percentage'] > 0) & (missing_analysis['Missing_Percentage'] <= 1)]
print(f"📊 Missing Value Categories:")
print(f"   🔴 High missing (>10%): {len(high_missing)} columns")
print(f"   🟡 Medium missing (1-10%): {len(medium_missing)} columns")
print(f"   🟢 Low missing (<1%): {len(low_missing)} columns")
if len(high_missing) > 0:
    print(f"\n🔴 High Missing Columns:")
    for _, row in high_missing.iterrows():
        print(f"   {row['Column']:<25}: {row['Missing_Percentage']:5.1f}% missing")
imputation_strategy = {}
for col in df.columns:
    missing_pct = (df[col].isnull().sum() / len(df)) * 100
    
    if missing_pct == 0:
        imputation_strategy[col] = 'no_action'
    elif missing_pct > 25:
        imputation_strategy[col] = 'flag_and_median'
    elif missing_pct > 10:
        if col in esg_columns:
            imputation_strategy[col] = 'knn_imputation'
        else:
            imputation_strategy[col] = 'median_mode'
    elif missing_pct > 1:
        imputation_strategy[col] = 'median_mode'
    else:
        imputation_strategy[col] = 'forward_fill'
print(f"\n📋 Imputation Strategy Summary:")
strategy_counts = pd.Series(list(imputation_strategy.values())).value_counts()
for strategy, count in strategy_counts.items():
    print(f"   {strategy}: {count} columns")

In [ ]:
print("🔧 STEP 6: IMPLEMENTING MISSING VALUE IMPUTATION")
print("=" * 50)
df_imputed = df.copy()
for col, strategy in imputation_strategy.items():
    if strategy == 'no_action':
        continue
    
    missing_before = df_imputed[col].isnull().sum()
    
    if strategy == 'forward_fill':
        if df_imputed[col].dtype in ['object']:
            df_imputed[col].fillna(method='ffill', inplace=True)
            df_imputed[col].fillna('Unknown', inplace=True)
        else:
            df_imputed[col].fillna(df_imputed[col].median(), inplace=True)
    
    elif strategy == 'median_mode':
        if df_imputed[col].dtype in ['object']:
            mode_val = df_imputed[col].mode()
            fill_val = mode_val.iloc[0] if len(mode_val) > 0 else 'Unknown'
            df_imputed[col].fillna(fill_val, inplace=True)
        else:
            df_imputed[col].fillna(df_imputed[col].median(), inplace=True)
    
    elif strategy == 'flag_and_median':
        df_imputed[f'{col}_was_missing'] = df_imputed[col].isnull().astype(int)
        if df_imputed[col].dtype in ['object']:
            mode_val = df_imputed[col].mode()
            fill_val = mode_val.iloc[0] if len(mode_val) > 0 else 'Unknown'
            df_imputed[col].fillna(fill_val, inplace=True)
        else:
            df_imputed[col].fillna(df_imputed[col].median(), inplace=True)
    
    elif strategy == 'knn_imputation' and col in esg_columns:
        if df_imputed[col].count() > 0:
            esg_subset = df_imputed[esg_columns].select_dtypes(include=[np.number])
            if esg_subset.shape[1] > 1:
                try:
                    imputer = KNNImputer(n_neighbors=5)
                    esg_imputed = imputer.fit_transform(esg_subset)
                    
                    col_idx = esg_subset.columns.get_loc(col)
                    df_imputed[col] = esg_imputed[:, col_idx]
                except:
                    df_imputed[col].fillna(df_imputed[col].median(), inplace=True)
            else:
                df_imputed[col].fillna(df_imputed[col].median(), inplace=True)
    
    missing_after = df_imputed[col].isnull().sum()
    
    if missing_before > 0:
        print(f"   {col:<30}: {missing_before:3d} → {missing_after:3d} missing ({strategy})")

print(f"\n✅ Imputation completed!")
print(f"📊 Total missing values: {df.isnull().sum().sum()} → {df_imputed.isnull().sum().sum()}")
print(f"📊 Data completeness: {((df_imputed.shape[0] * df_imputed.shape[1] - df_imputed.isnull().sum().sum()) / (df_imputed.shape[0] * df_imputed.shape[1]) * 100):.1f}%")

In [ ]:
print("🔧 STEP 7: OUTLIER DETECTION AND TREATMENT")
print("=" * 45)
numeric_columns = df_imputed.select_dtypes(include=[np.number]).columns
outlier_summary = []
for col in numeric_columns:
    if df_imputed[col].count() > 0:
        Q1 = df_imputed[col].quantile(0.25)
        Q3 = df_imputed[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = df_imputed[(df_imputed[col] < lower_bound) | (df_imputed[col] > upper_bound)]
        outlier_count = len(outliers)
        outlier_percentage = (outlier_count / len(df_imputed)) * 100
        
        outlier_summary.append({
            'Column': col,
            'Outliers': outlier_count,
            'Percentage': outlier_percentage,
            'Lower_Bound': lower_bound,
            'Upper_Bound': upper_bound
        })
outlier_df = pd.DataFrame(outlier_summary)
outlier_df = outlier_df.sort_values('Percentage', ascending=False)
print("📊 Outlier Analysis:")
for _, row in outlier_df.head(10).iterrows():
    if row['Outliers'] > 0:
        print(f"   {row['Column']:<30}: {row['Outliers']:3d} outliers ({row['Percentage']:4.1f}%)")
print(f"\n🎯 Outlier Treatment Strategy:")
for col in numeric_columns:
    if col in outlier_df[outlier_df['Percentage'] > 5]['Column'].values:
        if 'Employee' in col:
            cap_value = df_imputed[col].quantile(0.99)
            outlier_count = (df_imputed[col] > cap_value).sum()
            df_imputed[col] = np.where(df_imputed[col] > cap_value, cap_value, df_imputed[col])
            print(f"   {col}: Capped {outlier_count} values at 99th percentile ({cap_value:,.0f})")
        elif col in esg_columns:
            outlier_count = len(outlier_df[outlier_df['Column'] == col])
            if outlier_count > 0:
                print(f"   {col}: Keeping {outlier_df[outlier_df['Column']==col]['Outliers'].iloc[0]} outliers (legitimate ESG risk variation)")
        else:
            outlier_info = outlier_df[outlier_df['Column'] == col].iloc[0]
            if outlier_info['Outliers'] > 0:
                print(f"   {col}: Flagged {outlier_info['Outliers']} outliers for review")

print(f"\n✅ Outlier treatment completed!")

In [ ]:
print("🔧 STEP 8: FEATURE ENGINEERING")
print("=" * 35)
df_featured = df_imputed.copy()
if len([col for col in esg_columns if col in df_featured.columns]) >= 3:
    if all(col in df_featured.columns for col in ['Environment Risk Score', 'Social Risk Score', 'Governance Risk Score']):
        df_featured['ESG_Component_Balance'] = df_featured[['Environment Risk Score', 'Social Risk Score', 'Governance Risk Score']].std(axis=1)
        print("✅ Created ESG_Component_Balance (std of E, S, G scores)")
        
        df_featured['ESG_Max_Component'] = df_featured[['Environment Risk Score', 'Social Risk Score', 'Governance Risk Score']].max(axis=1)
        df_featured['ESG_Min_Component'] = df_featured[['Environment Risk Score', 'Social Risk Score', 'Governance Risk Score']].min(axis=1)
        print("✅ Created ESG_Max_Component and ESG_Min_Component")
if 'Full Time Employees' in df_featured.columns:
    df_featured['Employee_Size_Category'] = pd.cut(
        df_featured['Full Time Employees'], 
        bins=[0, 1000, 10000, 50000, float('inf')], 
        labels=['Small', 'Medium', 'Large', 'Enterprise'],
        include_lowest=True
    )
    print("✅ Created Employee_Size_Category (Small/Medium/Large/Enterprise)")
    
    df_featured['Log_Employees'] = np.log1p(df_featured['Full Time Employees'])
    print("✅ Created Log_Employees (log-transformed employee count)")
if 'Total ESG Risk score' in df_featured.columns:
    esg_risk_mean = df_featured['Total ESG Risk score'].mean()
    df_featured['ESG_Risk_Above_Average'] = (df_featured['Total ESG Risk score'] > esg_risk_mean).astype(int)
    print(f"✅ Created ESG_Risk_Above_Average (threshold: {esg_risk_mean:.1f})")
    
    df_featured['ESG_Risk_Category'] = pd.cut(
        df_featured['Total ESG Risk score'],
        bins=[0, 15, 25, 35, float('inf')],
        labels=['Low', 'Medium', 'High', 'Severe'],
        include_lowest=True
    )
    print("✅ Created ESG_Risk_Category (Low/Medium/High/Severe)")
if 'Sector' in df_featured.columns:
    sector_risk_avg = df_featured.groupby('Sector')['Total ESG Risk score'].mean()
    df_featured['Sector_Risk_Average'] = df_featured['Sector'].map(sector_risk_avg)
    df_featured['ESG_vs_Sector_Average'] = df_featured['Total ESG Risk score'] - df_featured['Sector_Risk_Average']
    print("✅ Created Sector_Risk_Average and ESG_vs_Sector_Average")
if 'ESG Risk Percentile' in df_featured.columns and 'ESG_Risk_Percentile_Numeric' in df_featured.columns:
    df_featured['High_Risk_Percentile'] = (df_featured['ESG_Risk_Percentile_Numeric'] > 75).astype(int)
    print("✅ Created High_Risk_Percentile (>75th percentile flag)")
new_features = [col for col in df_featured.columns if col not in df_imputed.columns]
print(f"\n📊 Feature Engineering Summary:")
print(f"   Original features: {len(df_imputed.columns)}")
print(f"   New features: {len(new_features)}")
print(f"   Total features: {len(df_featured.columns)}")
if new_features:
    print(f"\n🆕 New features created:")
    for i, feature in enumerate(new_features, 1):
        print(f"   {i}. {feature}")

In [ ]:
print("🔧 STEP 9: ENCODING CATEGORICAL VARIABLES")
print("=" * 45)
df_encoded = df_featured.copy()
categorical_cols = df_encoded.select_dtypes(include=['object', 'category']).columns
categorical_cols = [col for col in categorical_cols if col not in ['Symbol', 'Name', 'Address', 'Description']]
print(f"📊 Categorical columns to encode: {len(categorical_cols)}")
if len(categorical_cols) > 0:
    for col in categorical_cols:
        print(f"\n🔤 Processing {col}:")
        unique_count = df_encoded[col].nunique()
        print(f"   Unique values: {unique_count}")
        
        if unique_count <= 10:
            dummies = pd.get_dummies(df_encoded[col], prefix=f'{col}', dummy_na=True)
            df_encoded = pd.concat([df_encoded, dummies], axis=1)
            print(f"   ✅ One-hot encoded into {len(dummies.columns)} columns")
        else:
            le = LabelEncoder()
            df_encoded[f'{col}_Encoded'] = le.fit_transform(df_encoded[col].fillna('Unknown'))
            print(f"   ✅ Label encoded as {col}_Encoded")
        
        print(f"   📝 Keeping original {col} for reference")
print(f"\n📊 Encoding Summary:")
print(f"   Columns after encoding: {len(df_encoded.columns)}")
new_encoded_cols = [col for col in df_encoded.columns if col not in df_featured.columns]
print(f"   New encoded columns: {len(new_encoded_cols)}")

In [ ]:
print("🔧 STEP 10: FEATURE SCALING AND NORMALIZATION")
print("=" * 50)
df_final = df_encoded.copy()
numeric_cols_to_scale = df_final.select_dtypes(include=[np.number]).columns
id_cols = ['Symbol', 'Name', 'Address', 'Description']
exclude_from_scaling = ['Symbol', 'Name', 'Address', 'Description'] + \
                      [col for col in df_final.columns if '_was_missing' in col or 
                       col.endswith('_Encoded') or 
                       col.startswith(tuple(['Sector_', 'Industry_', 'Controversy Level_', 'ESG Risk Level_']))]
scale_cols = [col for col in numeric_cols_to_scale if col not in exclude_from_scaling]
print(f"📊 Columns to scale: {len(scale_cols)}")
print(f"   ESG-related: {[col for col in scale_cols if any(kw in col.lower() for kw in ['esg', 'risk', 'score', 'environment', 'social', 'governance'])]}")
print(f"   Other numeric: {[col for col in scale_cols if not any(kw in col.lower() for kw in ['esg', 'risk', 'score', 'environment', 'social', 'governance'])]}")
if len(scale_cols) > 0:
    scaler = StandardScaler()
    df_final[scale_cols] = scaler.fit_transform(df_final[scale_cols])
    print(f"\n✅ Standardized {len(scale_cols)} numeric columns (mean=0, std=1)")
    
    scaling_stats = pd.DataFrame({
        'Mean': df_final[scale_cols].mean(),
        'Std': df_final[scale_cols].std(),
        'Min': df_final[scale_cols].min(),
        'Max': df_final[scale_cols].max()
    })
    print(f"\n📊 Post-scaling statistics:")
    print(f"   Mean range: {scaling_stats['Mean'].min():.3f} to {scaling_stats['Mean'].max():.3f}")
    print(f"   Std range: {scaling_stats['Std'].min():.3f} to {scaling_stats['Std'].max():.3f}")
else:
    print("❌ No columns found for scaling")

In [ ]:
print("📊 FINAL DATA QUALITY ASSESSMENT")
print("=" * 45)
print(f"🎯 Processing Summary:")
print(f"   Original shape: {df_raw.shape}")
print(f"   Final shape: {df_final.shape}")
print(f"   Rows added/removed: {df_final.shape[0] - df_raw.shape[0]:+d}")
print(f"   Columns added: {df_final.shape[1] - df_raw.shape[1]:+d}")
print(f"\n📈 Data Quality Metrics:")
original_completeness = ((df_raw.shape[0] * df_raw.shape[1] - df_raw.isnull().sum().sum()) / (df_raw.shape[0] * df_raw.shape[1]) * 100)
final_completeness = ((df_final.shape[0] * df_final.shape[1] - df_final.isnull().sum().sum()) / (df_final.shape[0] * df_final.shape[1]) * 100)
print(f"   Original completeness: {original_completeness:.1f}%")
print(f"   Final completeness: {final_completeness:.1f}%")
print(f"   Improvement: {final_completeness - original_completeness:+.1f}%")
print(f"\n🔢 Column Types:")
numeric_count = len(df_final.select_dtypes(include=[np.number]).columns)
categorical_count = len(df_final.select_dtypes(include=['object', 'category']).columns)
print(f"   Numeric columns: {numeric_count}")
print(f"   Categorical columns: {categorical_count}")
print(f"   Total columns: {len(df_final.columns)}")
remaining_missing = df_final.isnull().sum().sum()
if remaining_missing > 0:
    print(f"\n⚠️  Remaining missing values: {remaining_missing}")
    missing_cols = df_final.columns[df_final.isnull().any()].tolist()
    for col in missing_cols[:5]:
        missing_count = df_final[col].isnull().sum()
        print(f"   {col}: {missing_count} missing")
else:
    print(f"\n✅ No missing values remaining!")
print(f"\n🎯 Ready for Machine Learning:")
ml_ready_cols = [col for col in df_final.columns 
                if col not in ['Symbol', 'Name', 'Address', 'Description'] 
                and df_final[col].dtype in [np.number, 'int64', 'float64']]
print(f"   ML-ready numeric features: {len(ml_ready_cols)}")
print(f"   Memory usage: {df_final.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
print("💾 SAVING PROCESSED DATA")
print("=" * 30)
os.makedirs(output_path, exist_ok=True)
processed_file_path = os.path.join(output_path, 'processed.csv')
metadata_file_path = os.path.join(output_path, 'processing_metadata.txt')
try:
    df_final.to_csv(processed_file_path, index=False)
    print(f"✅ Processed data saved to: {processed_file_path}")
    print(f"📊 File size: {os.path.getsize(processed_file_path) / 1024**2:.2f} MB")
    
    metadata = f"""ESG SUSTAINABILITY DATA PROCESSING METADATA
Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
====================================================

ORIGINAL DATA:
- Shape: {df_raw.shape}
- Completeness: {original_completeness:.1f}%
- Missing values: {df_raw.isnull().sum().sum():,}

PROCESSED DATA:
- Shape: {df_final.shape}
- Completeness: {final_completeness:.1f}%
- Missing values: {df_final.isnull().sum().sum():,}
- ML-ready features: {len(ml_ready_cols)}

PROCESSING STEPS:
1. ✅ Basic cleaning (duplicates, symbols)
2. ✅ Employee data standardization
3. ✅ ESG scores preprocessing
4. ✅ Categorical data standardization
5. ✅ Missing value imputation
6. ✅ Outlier detection and treatment
7. ✅ Feature engineering
8. ✅ Categorical encoding
9. ✅ Feature scaling/normalization
10. ✅ Final quality assessment

NEW FEATURES CREATED:
{chr(10).join([f"- {feature}" for feature in new_features])}

ENCODED COLUMNS:
{chr(10).join([f"- {col}" for col in new_encoded_cols])}

IMPUTATION STRATEGIES USED:
{chr(10).join([f"- {strategy}: {count} columns" for strategy, count in strategy_counts.items()])}

READY FOR:
- Machine Learning model training
- Statistical analysis
- Predictive modeling
- ESG risk assessment
"""

    with open(metadata_file_path, 'w', encoding='utf-8') as f:
        f.write(metadata)
    
    print(f"✅ Processing metadata saved to: {metadata_file_path}")
    
except Exception as e:
    print(f"❌ Error saving files: {e}")

print(f"\n🎉 DATA PREPROCESSING COMPLETED SUCCESSFULLY!")
print(f"📈 Dataset ready for machine learning and advanced analytics")
print(f"📁 Processed files location: {output_path}")
print(f"⏱️  Processing completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
print("🔍 PROCESSED DATA SAMPLE PREVIEW")
print("=" * 40)
print(f"📋 Final Dataset Shape: {df_final.shape}")
print(f"\n📊 First 3 rows of key columns:")
key_columns = ['Symbol', 'Name', 'Sector', 'Total ESG Risk score', 'ESG_Risk_Category', 'Employee_Size_Category']
display_cols = [col for col in key_columns if col in df_final.columns]
if len(display_cols) > 0:
    display(df_final[display_cols].head(3))
print(f"\n📈 Statistical Summary of Key ESG Metrics:")
esg_metrics = [col for col in df_final.columns if any(kw in col.lower() for kw in ['esg', 'risk', 'score']) and df_final[col].dtype in [np.number]]
if len(esg_metrics) > 0:
    display(df_final[esg_metrics[:6]].describe().round(3))
print(f"\n🎯 Column Categories:")
print(f"   📊 Numeric: {len(df_final.select_dtypes(include=[np.number]).columns)} columns")
print(f"   🔤 Categorical: {len(df_final.select_dtypes(include=['object']).columns)} columns")
print(f"   🆕 Engineered: {len(new_features)} columns")
print(f"   🎨 Encoded: {len(new_encoded_cols)} columns")
print(f"\n✅ Data preprocessing pipeline completed successfully!")
print(f"🚀 Ready for model training and advanced analytics")